# SQL Example

Get & plot data from a SQL DB.

* For drivers, this notebook uses pure Python ODBC (`pypyodbc`) and connects to Postgres
* Creates a table in the DB if does not exist

## Creds

* Create ODBC connection string
* We recommend changing `user`/`pwd` to ENV_VARS or files rather than visible hard-coded strings

In [56]:
user = "myadmin"
pwd = "mypassword"
server = "rds-test.zzz.us-west-2.rds.amazonaws.com:5432/postgres"
#import os
#user = os.environ['MY_USER']
#pwd = os.environ['MY_PWD']
#server = os.environ['MY_SERVER']

db_string = "postgres://" + user + ":" + pwd + "@" + server

In [57]:
import graphistry
#graphistry.register(key='MY_API_KEY', server='my.server.com')

## Optional: Install ODBC drivers for Postgres

In [ ]:
# Optional: Install a faster custom ODBC driver by logging in to jupyter docker as root:

# $ ssh -i key.pem ubuntu@my_public_ip
# ubuntu@my_private_ip $ docker exec -it -u root graphistry_notebook_1 bash
# root@601a8af7ea4c $ apt-get update
# root@601a8af7ea4c $ apt-get install unixodbc
# root@601a8af7ea4c $ pip3 install pyodbc

In [1]:
# If not already exists...

!pip3 install wheel -q
!pip3 install pypyodbc -q
!pip3 install sqlalchemy -q

## Connect to DB

In [34]:
import pandas as pd
import sqlalchemy as db
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, Float, String
from sqlalchemy.orm import sessionmaker

In [19]:
engine = create_engine(db_string)
Session = sessionmaker(bind=engine)
session = Session()

## Optional: Prepopulate table "my_table"

In [58]:
engine.table_names()

['my_table']

In [59]:
if not engine.dialect.has_table(engine, 'my_table'):  # If table don't exist, Create.
    metadata = MetaData(engine)
    mytable = Table('my_table', metadata,
          Column('Id', Integer, primary_key=True, nullable=False), 
          Column('Country', String),
          Column('Brand', String),
          Column('Price', Float))
    metadata.create_all()
    engine.execute(
        mytable.insert(),
            [{"Country": "c" + str(x % 2), "Brand": "b_" + str(x), "Price": x * 2} for x in range(0, 10)])          

## Get data

In [54]:
result = engine.execute("SELECT * FROM my_table LIMIT 10")
df = pd.DataFrame(result.fetchall(), columns=result.keys())
print('# rows', len(df))
df.sample(min(3, len(df)))

# rows 10


,Id,Country,Brand,Price
6,7,c0,b_6,12.0
5,6,c1,b_5,10.0
7,8,c1,b_7,14.0


## Plot

Several variants:
1. Treat each row & cell value as a node, and connect row<>cell values
2. Treat each cell value as an edge, and connect all cell values together when they occur on the same row
3. Treat each cell value as an edge, and specify which columns to to connect values together on

In [65]:
hg = graphistry.hypergraph(df, ['Id', 'Country', 'Brand'])
hg['graph'].plot()

# links 30
# events 10
# attrib entities 22


/usr/local/lib/python3.6/dist-packages/graphistry/hyper.py:171: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  nodes = pd.concat([entities, event_entities], ignore_index=True).reset_index(drop=True)


In [67]:
hg = graphistry.hypergraph(
    df, 
    ['Id', 'Country', 'Brand'],
    direct=True)
hg['graph'].plot()

# links 30
# events 10
# attrib entities 22


In [ ]:
hg = graphistry.hypergraph(
    df, 
    ['Id', 'Country', 'Brand'],
    direct=True,
    opts={
        'EDGES': {
            'Id': ['Country', 'Brand'],
            'Brand': ['Country']
        }
    })
hg['graph'].plot()